In [ ]:
import numpy as np
import cv2
from matplotlib import pyplot as plt

import session
# termination criteria
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((7*7,3), np.float32)
objp[:,:2] = np.mgrid[0:7,0:7].T.reshape(-1,2)

objp *= 3
# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.
sessionPath = "/Volumes/Public/Shared Files/KamerJelle/session-2022-03-04 11-41-13"
calibrationSession = session.Session().from_path(sessionPath)
plt.rcParams['figure.dpi'] = 300

for image in calibrationSession.imageTransforms:
    print("Calibrating", image.id)
    gray = cv2.cvtColor(cv2.imread(image.path), cv2.COLOR_BGR2GRAY)
    # Find the chess board corners
    ret, corners = cv2.findChessboardCorners(gray, (7, 7), None)

    #plt.imshow(gray)
    #plt.show()

    # If found, add object points, image points (after refining them)
    if ret:
        print("found Chessboard")
        objpoints.append(objp)

        corners2 = cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)
        imgpoints.append(corners2)
        #imageDraw = cv2.imread(image.path)
        #cv2.drawChessboardCorners(imageDraw, (7,7), corners2, ret)
        #plt.imshow(cv2.cvtColor(imageDraw, cv2.COLOR_BGR2RGB))
        #plt.title("Match3")
        #plt.show()

    # Calibrate camera
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

print(ret,"\n", mtx,"\n", dist,"\n", rvecs,"\n", tvecs)

print("intrinsic camera matrix: \n", mtx)

In [ ]:
mean_error = 0
for i in range(len(objpoints)):
    imgpoints2, _ = cv2.projectPoints(objpoints[i], rvecs[i], tvecs[i], mtx, dist)
    error = cv2.norm(imgpoints[i], imgpoints2, cv2.NORM_L2)/len(imgpoints2)
    mean_error += error
print( "total error: {}".format(mean_error/len(objpoints)) )

In [ ]:
calibrationSession.imageTransforms[0].fov *=1
print(calibrationSession.imageTransforms[0].get_camera_matrix())


## Accuracy test
Test the accuracy of the estimated camera matrix with the given coordinates of the cameras

In [ ]:
testSessionPath = "/Volumes/Data drive/Documents/Doctoraat Local/XR Paper Data/RefSessions/LivingLab/building big/session-2022-02-10"

import session
testSession = session.Session().from_path(testSessionPath)

In [ ]:
import session

path = "/Volumes/Data drive/Documents/Doctoraat Local/SfM_quality_evaluation/Benchmarking_Camera_Calibration_2008/castle-P19"

testSession = session.Session().benchMark_to_session(path)

In [ ]:
# set the matrix with the fov
imageFov = 57.6

for image in testSession.imageTransforms:
    image.fov = imageFov

testSession.convert_axis("y")

In [ ]:
# set the image with the full matrix
imageMatrix = mtx

for image in testSession.imageTransforms:
    image.cameraMatrix = imageMatrix

testSession.convert_axis("y")

In [ ]:
image1 = testSession.imageTransforms[0]
image2 = testSession.imageTransforms[1]

import cv2
from imagematch import ImageMatch
from matplotlib import pyplot as plt

# Displaying the matches
match1 = ImageMatch(image1, image2)
match1.find_matches()
match1.get_essential_matrix()
match1.get_reference_scaling_factor()
match1.triangulate(True)

match1Image = match1.draw_image_inliers()

plt.rcParams['figure.dpi'] = 300
plt.axis('off')
plt.imshow(cv2.cvtColor(match1Image, cv2.COLOR_BGR2RGB))
plt.title("Match1")
plt.show()

In [ ]:
#plot the 3 images and a mesh

import positioning3d as pos3d

R,t = match1.get_image2_pos()

pos3d.show_geometries([
    #testImage.get_camera_geometry(), 
    image1.get_camera_geometry(), 
    image2.get_camera_geometry(),
    pos3d.create_3d_camera(t,R, 1),
    #testSession.geometries[0].geometry,
    match1.get_point_cloud()
], True)